In [16]:
from langchain_core.language_models.llms import LLM
from langchain_core.messages import AIMessage
from transformers import AutoTokenizer, AutoConfig,AutoModel
from typing import Any, List, Optional, Dict, Iterator

class qwen(LLM):
    max_tokens: int = 8192
    do_sample:bool =True
    temperature: float = 0.3
    top_p:float=0.0
    tokenizer: object = None
    model:object=None
    history: List[Dict] = []
    
    def __init__(self):
        super().__init__()
        
    @property 
    def _llm_type(self):
        return "qwen"
      
    def load_model(self,modelPath=None):
        #配置分词器
        tokenizer=AutoTokenizer.from_pretrained(modelPath,trust_remote_code=True,use_fast=True)
        
        #加载模型
        model=AutoModel.from_pretrained(modelPath,trust_remote_code=True,device_map="auto")
        
        model=model.eval()
        
        self.model=model
        self.tokenizer=tokenizer
        
    def _call(self,prompt,config={},history=[]):
        return self.invoke(prompt,history)
    
    def invoke(self,prompt,history=[]):
        if not isinstance(prompt,str):            
            prompt=prompt.to_string()        
        response,history=self.model.chat(
            self.tokenizer,
            prompt,
            history=history,
            do_sample=self.do_sample,
            max_length=self.max_tokens,
            temperature=self.temperature
        )
    
    def stream(self,prompt,config={},history=[]):
        if not isinstance(prompt,str):            
            prompt=prompt.to_string() 
        preResponse=""
        for response,new_history in self.model.stream_chat():
            #self.history=new_history
            
            if preResponse=="":
                reseult=response
            else:
                result=response[len(preResponse):]
            preResponse=response
            yield result


In [17]:
llm=qwen()
#modelPath="YOur Model Path"
#llm.loadModel(modelPath)
llm.invoke("如何鼓励自己减肥")
